First, we import all modules and resources from source folder `src/`.

In [ ]:
# import modules from src
import src.config as config
from src.bot import *
from src.api import *

Then you can test any functions you want. For example, we make an test case to test `on_message()` activity handler.

In [ ]:

# ## Test the bot activity on_message()

# from botbuilder.core import TurnContext
# from unittest.mock import MagicMock, patch

# def mock_send_activity(message):
#     print("[mocked]", message)

# async def test_on_message():
#     # Mock the content of the context object
#     context = MagicMock(spec=TurnContext)
#     context.activity.channel_id = "mock_channel"
#     context.activity.from_property.id = "mock_user"

#     # Set the activity properties
#     context.activity.text = "Hello, test messages!!"
    
#     context.send_activity = MagicMock(side_effect=mock_send_activity)

#     try:
#         # Call the on_message() function
#         await on_message(context, None)
#     except Exception as e:
#         print("Exception: ", e)
    

# # Call the test function
# await test_on_message()

There are lot's of functions you can add to this basic echo bot.

In [ ]:
# Add planner
from dataclasses import field
from teams import AIHistoryOptions, AIOptions, AzureOpenAIPlanner, AzureOpenAIPlannerOptions, ConversationHistory, ConversationState, TempState, UserState, prompt_template, prompt_manager
from botbuilder.core import MemoryStorage
from botbuilder.schema import Activity, ActivityTypes, ChannelAccount, ConversationAccount
from unittest.mock import MagicMock 
config.AZURE_OPENAI_KEY = 'b9e28116eeff4b129db5155fd261a541' # Azure OpenAI API key
config.AZURE_OPENAI_MODEL_DEPLOYMENT_NAME = 'gpt-35-turbo-zihch' # Azure OpenAI model deployment name
config.AZURE_OPENAI_ENDPOINT = 'https://azure-openai-zihch.openai.azure.com/' # Azure OpenAI endpoint
planner = AzureOpenAIPlanner(
    AzureOpenAIPlannerOptions(
        config.AZURE_OPENAI_KEY,
        config.AZURE_OPENAI_MODEL_DEPLOYMENT_NAME,
        config.AZURE_OPENAI_ENDPOINT,
        prompt_folder="src/prompts",
    )
)

# mock the TurnContext object
context=MagicMock(spec=TurnContext)

# mock the TurnState object
temp=TempState(input='hi', history='', output='')
user=UserState(__key__='mock_key', channel=ChannelAccount())
conversation=ConversationState(
    __key__='mock_key',
    account=ConversationAccount(),
    channel=ChannelAccount(),
    history=ConversationHistory()
)
state=TurnState(
    temp=temp,
    user=user,
    conversation=conversation
)

# render prompt
prompt="chat"
prompt_templates = await planner._prompt_manager.render_prompt(context, state, prompt)
print(f"prompt_templates:\n{prompt_templates}")

# generate plan
plan = await planner.generate_plan(
    context, state, prompt, history_options=AIHistoryOptions(assistant_history_type="text")
)

print(f"plan:\n{plan}")

app = Application[TurnState]( 
    ApplicationOptions(
        auth=BotFrameworkAdapterSettings(
            app_id=config.app_id,
            app_password=config.app_password
        ),
        ai=AIOptions(
            planner=planner,
        ),
        storage=MemoryStorage()
    )
)

